In [49]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

import os 
import cv2


data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_CHANNELS = 1
batchsize=1

TRAIN_DATA_DIR='data/train_data/x/'
TRAIN_MASK_DIR='data/train_data/y/'
TEST_DATA_DIR='data/train_data/x/'
TEST_MASK_DIR='data/train_data/y/'

In [50]:
seed = 1
# image_datagen.fit(images, augment=True, seed=seed)
# mask_datagen.fit(masks, augment=True, seed=seed)

image_generator = image_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    class_mode=None,
    seed=seed,
    batch_size=batchsize)

mask_generator = mask_datagen.flow_from_directory(
    TRAIN_MASK_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    class_mode=None,
    seed=seed,
    batch_size=batchsize,
    color_mode='grayscale')

Found 33 images belonging to 1 classes.
Found 32 images belonging to 1 classes.


In [51]:
nb_samples = len(image_generator.filenames)
print(nb_samples)

33


In [52]:


# Build U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 512, 512, 1)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_77 (Conv2D)              (None, 512, 512, 16) 160         lambda_5[0][0]                   
__________________________________________________________________________________________________
dropout_37 (Dropout)            (None, 512, 512, 16) 0           conv2d_77[0][0]                  
__________________________________________________________________________________________________
conv2d_78 

In [54]:
listaFicheros = [os.path.join(TRAIN_DATA_DIR, 'x/')+i for i in os.listdir(os.path.join(TRAIN_DATA_DIR, 'x/'))] 
x = []
y = []
for image in listaFicheros:
        print(image)
        x.append(cv2.resize(cv2.imread(image), (IMG_WIDTH,IMG_HEIGHT), interpolation=cv2.INTER_CUBIC))
        
image_datagen.fit(x)

listaFicheros = [os.path.join(TRAIN_MASK_DIR, 'y/')+i for i in os.listdir(os.path.join(TRAIN_MASK_DIR, 'y/'))] 
for image in listaFicheros:
        print(image)
        y.append(cv2.resize(cv2.imread(image), (IMG_WIDTH,IMG_HEIGHT), interpolation=cv2.INTER_CUBIC))
        
mask_datagen.fit(y)

data/train_data/x/x/1.tif
data/train_data/x/x/10.tif
data/train_data/x/x/11.tif
data/train_data/x/x/12.tif
data/train_data/x/x/13.tif
data/train_data/x/x/14.tif
data/train_data/x/x/15.tif
data/train_data/x/x/16.tif
data/train_data/x/x/17.tif
data/train_data/x/x/18.tif
data/train_data/x/x/19.tif
data/train_data/x/x/2.tif
data/train_data/x/x/20.tif
data/train_data/x/x/21.tif
data/train_data/x/x/22.tif
data/train_data/x/x/23.tif
data/train_data/x/x/24.tif
data/train_data/x/x/25.tif
data/train_data/x/x/26.tif
data/train_data/x/x/27.tif
data/train_data/x/x/28.tif
data/train_data/x/x/29.tif
data/train_data/x/x/3.tif
data/train_data/x/x/30.tif
data/train_data/x/x/31.tif
data/train_data/x/x/32.tif
data/train_data/x/x/33.tif
data/train_data/x/x/4.tif
data/train_data/x/x/5.tif
data/train_data/x/x/6.tif
data/train_data/x/x/7.tif
data/train_data/x/x/8.tif
data/train_data/x/x/9.tif
data/train_data/y/y/1.tif
data/train_data/y/y/10.tif
data/train_data/y/y/11.tif
data/train_data/y/y/12.tif
data/train_

In [55]:
train_generator = zip(image_generator, mask_generator)

model.fit_generator(train_generator, steps_per_epoch=nb_samples/batchsize, epochs=100, verbose=1)

Epoch 1/100


ValueError: non-broadcastable output operand with shape (512,512,1) doesn't match the broadcast shape (512,512,3)

['data/train_data/x/x']